# Cell

A `@gf.cell` is a decorator for functions that return a Component. Make sure you add the `@gf.cell` decorator to each function that returns a Component so you avoid having multiple components with the same name.

Why do you need to add the `@gf.cell` decorator?

- In GDS each component must have a unique name. Ideally the name is also consistent from run to run, in case you want to merge GDS files that were created at different times or computers.
- Two components stored in the GDS file cannot have the same name. They need to be references (instances) of the same component. See `References tutorial`. That way we only have to store the component in memory once and all the references are just pointers to that component.

What does the `@gf.cell` decorator does?

1. Gives the component a unique name depending on the parameters that you pass to it.
2. Creates a cache of components where we use the name as the key. The first time the function runs, the cache stores the component, so the second time, you get the component directly from the cache, so you don't create the same component twice.


A decorator is a function that runs over a function, so when you do.

```python
@gf.cell
def mzi_with_bend():
    c = gf.Component()
    mzi = c << gf.components.mzi()
    bend = c << gf.components.bend_euler()
    return c
```
it's equivalent to

```python
def mzi_with_bend():
    c = gf.Component()
    mzi = c << gf.components.mzi()
    bend = c << gf.components.bend_euler(radius=radius)
    return c


mzi_with_bend_decorated = gf.cell(mzi_with_bend)
```

Lets see how it works.

In [ ]:
import gdsfactory as gf


def mzi_with_bend(radius: float = 10.0) -> gf.Component:
    c = gf.Component()
    mzi = c << gf.components.mzi()
    bend = c << gf.components.bend_euler(radius=radius)
    bend.connect("o1", mzi.ports["o2"])
    return c


c = mzi_with_bend()
print(f"this cell {c.name!r} does NOT get automatic name")
f = c.plot()

In [ ]:
@gf.cell
def mzi_with_bend(radius: float = 10.0) -> gf.Component:
    c = gf.Component()
    mzi = c << gf.components.mzi()
    bend = c << gf.components.bend_euler(radius=radius)
    bend.connect("o1", mzi.ports["o2"])
    return c


c = mzi_with_bend(radius=12)
print(f"this cell {c.name!r} gets automatic name thanks to the `cell` decorator")
f = c.plot()

In [ ]:
@gf.cell
def wg(length=10, width=1, layer=(1, 0)):
    print(f"BUILDING waveguide, length = {length}, width = {width} ")
    c = gf.Component()
    c.add_polygon([(0, 0), (length, 0), (length, width), (0, width)], layer=layer)
    c.add_port(
        name="o1", center=[0, width / 2], width=width, orientation=180, layer=layer
    )
    c.add_port(
        name="o2", center=[length, width / 2], width=width, orientation=0, layer=layer
    )
    return c


# See how the cells get the name from the parameters that you pass them
c = wg()
print(c)

print("second time you get this cell from the cache")
c = wg()
print(c)

print("If you call the cell with different parameters, the cell gets a different name")
c = wg(width=0.5)
print(c)

In [ ]:
# Sometimes when you are changing the inside code of the function, you need to flush the cache
# You can pass an empty cache and then remove that once you are happy with the solution
@gf.cell(cache={})
def wg(length=10, width=1, layer=(1, 0)):
    print("BUILDING waveguide without CACHE")
    c = gf.Component()
    c.add_polygon([(0, 0), (length, 0), (length, width), (0, width)], layer=layer)
    c.add_port(
        name="o1", center=[0, width / 2], width=width, orientation=180, layer=layer
    )
    c.add_port(
        name="o2", center=[length, width / 2], width=width, orientation=0, layer=layer
    )
    return c


c = wg()
c

## Info

Together with the GDS file that you send to the foundry you can also store info settings.

In [ ]:
# thanks to `gf.cell` you can also add any metadata `info` relevant to the cell
c = wg(length=3)
c.info["polarization"] = "te"
c.info["wavelength"] = 1.55
c.info

## Cache

To avoid that 2 exact cells are not references of the same cell the `cell` decorator has a cache where if a component has already been built it will return the component from the cache

In [ ]:
@gf.cell
def wg(length=10, width=1):
    c = gf.Component()
    c.add_polygon([(0, 0), (length, 0), (length, width), (0, width)], layer=(1, 0))
    print("BUILDING waveguide")
    return c


wg1 = wg()  # cell builds a straight
print(wg1)

In [ ]:
wg2 = (
    wg()
)  # cell returns the same straight as before without having to run the function
print(wg2)